# Conversão de Radiação (Rs → n)

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# -*- coding: utf-8 -*-
"""
Conversão de Rs (MJ/m²/dia) -> n (horas de sol) usando Angström–Prescott,
com cálculo de N e Ra diários (FAO-56).

Percorre recursivamente todas as pastas em:
    E:\CLIMBRA_SSP245\rss_input

Mantém a mesma estrutura de pastas em:
    E:\CLIMBRA_SSP245\rss_output

Formato de entrada (CSV, separador ";"):
    dia;mes;ano;valor

Nome de arquivo (exemplo):
    ACCESS-ESM1-5-rss-ssp245_-25.125_-49.125.csv
"""

import re
from pathlib import Path

import numpy as np
import pandas as pd

# ======================================================
# CONFIGURAÇÕES
# ======================================================

INPUT_DIR = RAW_DIR / 'rss_input'
OUTPUT_DIR = OUT_DIR / 'rss_output'

# regex para extrair lat/lon do nome do arquivo
# ..._ -25.125 _ -49.125 .csv
LATLON_REGEX = r"_(?P<lat>-?\d+(?:\.\d+)?)_(?P<lon>-?\d+(?:\.\d+)?)\.csv$"

# Coeficientes de Angström–Prescott (padrão FAO-56 genérico)
A_ANGSTROM = 0.25
B_ANGSTROM = 0.50

# Se True, limita n entre 0 e N
CLIP_N = True

# Sufixo a ser adicionado no nome do arquivo de saída
OUTPUT_SUFFIX = "_n"

# ======================================================
# FUNÇÕES ASTRONÔMICAS (FAO-56)
# ======================================================

def solar_declination(J):
    """Declinação solar (radianos) – Allen et al. (1998) FAO-56."""
    return 0.409 * np.sin((2.0 * np.pi / 365.0) * J - 1.39)

def inverse_relative_distance_earth_sun(J):
    """Distância relativa Terra–Sol (adimensional)."""
    return 1.0 + 0.033 * np.cos((2.0 * np.pi / 365.0) * J)

def sunset_hour_angle(phi_rad, delta):
    """Ângulo horário ao pôr do sol (radianos)."""
    x = -np.tan(phi_rad) * np.tan(delta)
    x = np.clip(x, -1.0, 1.0)
    return np.arccos(x)

def day_length_hours(lat_deg, J):
    """Duração máxima do dia N (horas), para latitude (graus) e dia juliano J."""
    phi = np.radians(lat_deg)
    delta = solar_declination(J)
    omega_s = sunset_hour_angle(phi, delta)
    N = 24.0 / np.pi * omega_s
    return N

def extraterrestrial_radiation_MJm2d(lat_deg, J):
    """
    Radiação extraterrestre diária Ra (MJ/m²/dia) – FAO-56.
    Ra = (24*60/pi) * Gsc * dr * [ωs*sinφ*sinδ + cosφ*cosδ*sinωs]
    Gsc = 0.0820 MJ·m⁻²·min⁻¹
    """
    Gsc = 0.0820
    dr = inverse_relative_distance_earth_sun(J)
    phi = np.radians(lat_deg)
    delta = solar_declination(J)
    omega_s = sunset_hour_angle(phi, delta)

    Ra = (24.0 * 60.0 / np.pi) * Gsc * dr * (
        omega_s * np.sin(phi) * np.sin(delta)
        + np.cos(phi) * np.cos(delta) * np.sin(omega_s)
    )
    Ra = np.maximum(Ra, 0.0)
    return Ra

# ======================================================
# SUPORTE
# ======================================================

def parse_lat_lon_from_filename(filename: str):
    """
    Extrai latitude e longitude do nome do arquivo usando LATLON_REGEX.
    """
    m = re.search(LATLON_REGEX, filename)
    if not m:
        raise ValueError(f"Não foi possível extrair lat/lon do nome: {filename}")
    lat = float(m.group("lat"))
    lon = float(m.group("lon"))  # não usamos a lon aqui, mas já fica disponível
    return lat, lon

def process_file(path_in: Path, base_in: Path, base_out: Path):
    """
    Processa um único arquivo CSV:
      - lê dia/mes/ano/valor (Rs)
      - monta datetime
      - calcula J, N, Ra
      - calcula n (horas de sol)
      - salva CSV de saída na pasta correspondente em base_out
    """
    print(f"Processando: {path_in}")

    # Lê CSV com separador ';'
    df = pd.read_csv(path_in, sep=";")

    # Verifica colunas mínimas
    for col in ["dia", "mes", "ano", "valor"]:
        if col not in df.columns:
            raise KeyError(f"Coluna obrigatória '{col}' não encontrada em {path_in.name}")

    # Cria coluna de data
    df["data"] = pd.to_datetime(
        dict(year=df["ano"], month=df["mes"], day=df["dia"]),
        errors="coerce"
    )
    df = df.dropna(subset=["data"]).copy()

    # Dia juliano
    J = df["data"].dt.dayofyear.astype(int)

    # Latitude a partir do nome do arquivo
    lat, lon = parse_lat_lon_from_filename(path_in.name)

    # Rs já está em MJ/m²/dia
    Rs = df["valor"].astype(float)

    # Calcula N e Ra
    N_h   = day_length_hours(lat, J)
    Ra_MJ = extraterrestrial_radiation_MJm2d(lat, J)

    # Proteção para Ra=0
    Ra_safe = np.where(Ra_MJ > 0, Ra_MJ, np.nan)
    ratio = Rs / Ra_safe

    # Angström–Prescott: Rs/Ra = a + b * (n/N)
    # => n = N * ((Rs/Ra) - a) / b
    n_h = N_h * ((ratio - A_ANGSTROM) / B_ANGSTROM)

    if CLIP_N:
        n_h_clip = np.clip(n_h, 0.0, N_h)
    else:
        n_h_clip = n_h

    # Monta DataFrame de saída
    df_out = df.copy()
    df_out["Ra_MJm2d"]    = Ra_MJ
    df_out["N_h"]         = N_h
    df_out["Rs_MJm2d"]    = Rs
    df_out["Rs_Ra_ratio"] = ratio
    df_out["n_h"]         = n_h
    df_out["n_h_clip"]    = n_h_clip

    # Define pasta de saída preservando a estrutura relativa
    rel_parent = path_in.relative_to(base_in).parent   # ex: ACCESS-ESM1-5-rss-ssp245
    out_dir = base_out / rel_parent
    out_dir.mkdir(parents=True, exist_ok=True)

    # Nome de saída
    out_name = path_in.stem + OUTPUT_SUFFIX + path_in.suffix
    path_out = out_dir / out_name

    # Salva com o mesmo separador ';'
    df_out.to_csv(path_out, sep=";", index=False)

    print(f"  -> Salvo em: {path_out}")

def main():
    base_in = Path(INPUT_DIR)
    base_out = Path(OUTPUT_DIR)
    base_out.mkdir(parents=True, exist_ok=True)

    # rglob encontra todos os .csv em subpastas
    files = sorted(base_in.rglob("*.csv"))
    if not files:
        print(f"Nenhum arquivo .csv encontrado em {base_in}")
        return

    print(f"Encontrados {len(files)} arquivo(s) para processar.\n")

    for i, f in enumerate(files, start=1):
        try:
            print(f"[{i}/{len(files)}]")
            process_file(f, base_in, base_out)
        except Exception as e:
            print(f"ERRO ao processar {f}: {e}\n")

if __name__ == "__main__":
    main()